In [1]:
import pandas as pd

In [2]:
label_to_int = {
    "winner_model_a": 0,
    "winner_model_b": 1,
    "tie": 2
}

In [10]:
df = pd.read_csv("../data/train.csv")
print(df.columns)
df["left"] = df.apply(lambda row: f"Prompt: {row["prompt"]}\n\nResponse: {row["response_a"][0]}", axis=1)
df["right"] = df.apply(lambda row: f"Prompt: {row["prompt"]}\n\nResponse: {row["response_b"][0]}", axis=1)


df["winner"] = df.apply(
        lambda row: "winner_model_a" if row["winner_model_a"] == 1 else "winner_model_b" if row["winner_model_b"] == 1 else "tie",
        axis=1
    )

df["y"] = df["winner"].map(label_to_int)
df = df[["left", "right", "y"]]

Index(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie'],
      dtype='object')


In [11]:
df

,left,right,y
0,"Prompt: [""Is it morally right to try to have a...","Prompt: [""Is it morally right to try to have a...",0
1,"Prompt: [""What is the difference between marri...","Prompt: [""What is the difference between marri...",1
2,"Prompt: [""explain function calling. how would ...","Prompt: [""explain function calling. how would ...",2
3,"Prompt: [""How can I create a test set for a ve...","Prompt: [""How can I create a test set for a ve...",0
4,"Prompt: [""What is the best way to travel from ...","Prompt: [""What is the best way to travel from ...",1
...,...,...,...
57472,"Prompt: [""A simple mnemonic for \u03c0:\n\""How...","Prompt: [""A simple mnemonic for \u03c0:\n\""How...",0
57473,"Prompt: [""In python, implement a naive Bayes w...","Prompt: [""In python, implement a naive Bayes w...",0
57474,"Prompt: [""is it unethical to work on building ...","Prompt: [""is it unethical to work on building ...",0
57475,"Prompt: [""If a bait contains 0,0025% bromadiol...","Prompt: [""If a bait contains 0,0025% bromadiol...",1
